# Project 1
## The Net Wars: Attack of the Bots
### Advanced Machine Learning - MECD (2022/2023)

### Autores

- Duarte Meneses - 2019216949
- Patricia Costa - 2019213995

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
# Just to make plots look better
plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams['axes.grid'] = True
plt.style.use('fivethirtyeight')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['lines.linewidth'] = 3

|Class  |Label |
| ----- | ---- | 
|Normal	|0     |
|Dos	|1     |
|R2L	|2     |
|U2R	|3     |
|Probe	|4     |

In [3]:
PATH_TO_DATA = 'dataset/'

test_students = pd.read_csv(PATH_TO_DATA + 'test_students.csv')

train_students = pd.read_csv(PATH_TO_DATA + 'train_students.csv')

In [4]:
train_students = pd.get_dummies(train_students, columns=['protocol_type'], drop_first=True)
train_students = pd.get_dummies(train_students, columns=['service'], drop_first=True)
train_students= pd.get_dummies(train_students, columns=['flag'], drop_first=True)


In [10]:
print(train_students)

        duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
0              1       2599        293     0               0       0    0   
1              0          0          0     0               0       0    0   
2              0        280      13254     0               0       0    0   
3              0        230       1582     0               0       0    0   
4              0        218        483     0               0       0    0   
...          ...        ...        ...   ...             ...     ...  ...   
103956         0          0          0     0               0       0    0   
103957         0          0          0     0               0       0    0   
103958         0          0         83     0               0       0    0   
103959        50        226       2615     0               0       0    1   
103960         0         43         70     0               0       0    0   

        num_failed_logins  logged_in  num_compromised  ...  flag_REJ  \
0  

In [5]:
train_students.replace('normal', 0, inplace=True)
train_students.replace('Dos', 1, inplace=True)
train_students.replace('R2L', 2, inplace=True)
train_students.replace('U2R', 3, inplace=True)
train_students.replace('Probe', 4, inplace=True)


In [6]:
y_train = train_students['attack_type']
X_train = train_students.drop('attack_type', axis=1)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_t, y_train, y_t = train_test_split(X_train, y_train, test_size=0.4, random_state=42)
X_test, X_validation, y_test, y_validation = train_test_split(X_t, y_t, test_size=0.5, random_state=42)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier

lr = LogisticRegression(max_iter=1000)
svc = SVC()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
nb = GaussianNB()

voting = VotingClassifier(estimators=[
            ('lr', lr),
            ('svc', svc),
            ('knn', knn),
            ('dt', dt),
            ('nb', nb) ],
            voting='hard')

lista_classifiers = [lr, svc, knn, dt, nb, voting]

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

for cl in lista_classifiers:
    cl.fit(X_train, y_train)
    y_pred = cl.predict(X_validation)
    print("Accuracy:")
    print(cl.__class__.__name__, accuracy_score(y_validation, y_pred))
    print("F1-score:")
    print(cl.__class__.__name__, f1_score(y_validation, y_pred, average='micro'))
    print("Precision:")
    print(cl.__class__.__name__, precision_score(y_validation, y_pred, average='micro'))
    print("Recall:")
    print(cl.__class__.__name__, recall_score(y_validation, y_pred, average='micro'))

c:\Users\emanu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy:
LogisticRegression 0.7980570384263935
F1-score:
LogisticRegression 0.7980570384263935
Precision:
LogisticRegression 0.7980570384263935
Recall:
LogisticRegression 0.7980570384263935
Accuracy:
SVC 0.5192131967489059
F1-score:
SVC 0.5192131967489059
Precision:
SVC 0.5192131967489059
Recall:
SVC 0.5192131967489059
Accuracy:
KNeighborsClassifier 0.981147501563026
F1-score:
KNeighborsClassifier 0.981147501563026
Precision:
KNeighborsClassifier 0.981147501563026
Recall:
KNeighborsClassifier 0.981147501563026
Accuracy:
DecisionTreeClassifier 0.9928822199778772
F1-score:
DecisionTreeClassifier 0.9928822199778772
Precision:
DecisionTreeClassifier 0.9928822199778772
Recall:
DecisionTreeClassifier 0.9928822199778772
Accuracy:
GaussianNB 0.4274034530851729
F1-score:
GaussianNB 0.4274034530851729
Precision:
GaussianNB 0.4274034530851729
Recall:
GaussianNB 0.4274034530851729


c:\Users\emanu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy:
VotingClassifier 0.8644736209301207
F1-score:
VotingClassifier 0.8644736209301207
Precision:
VotingClassifier 0.8644736209301207
Recall:
VotingClassifier 0.8644736209301207
